In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import re

word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

[nltk_data] Downloading package punkt to /Users/sunhokim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sunhokim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
con_df = pd.read_csv('concrete.csv', delimiter = ',')
con_df.set_index('word', inplace=True)

In [17]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new
def summarize_sentences(sentences, n):
  if len(sentences) <= 2:
    return []
  clean_sentences = [{'id': sen['id'], 'content': sen['content'].replace("[^a-zA-Z]", " ")} for sen in sentences]
  clean_sentences = [{'id': s['id'], 'content': s['content'].lower()} for s in clean_sentences]
  clean_sentences = [{'id': s['id'], 'content': remove_stopwords(s['content'].split())} for s in clean_sentences]
  sentence_vectors = []
  for d in clean_sentences:
    i = d['content']
    if len(i) != 0:
      v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
      v = np.zeros((100,))
    sentence_vectors.append(v)

  sim_mat = np.zeros([len(sentences), len(sentences)])
  from sklearn.metrics.pairwise import cosine_similarity
  for i in range(len(sentences)):
    for j in range(len(sentences)):
      if i != j:
        sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
  import networkx as nx
  try:
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
  except:
    return []
  ranked_sentences = sorted(((scores[i],s['id'], s['content']) for i,s in enumerate(sentences)), reverse=True)
  return [{'id': ranked_sentences[i][1], 'content': ranked_sentences[i][2]} for i in range(min(n,len(ranked_sentences)))]


from nltk.corpus import wordnet
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gorani.models.quiz import Quiz, WordItem, Chapter
import simplejson
import re
book = None
lemmatizer = WordNetLemmatizer() 
with open('book2.book', 'rb') as f:
  book = simplejson.load(f)
fdist = FreqDist()
for chap in book['chapters']:
  for item in chap['items']:
    words = word_tokenize(item['content'])
    words = pd.Series(words).str.replace("[^a-zA-Z]", " ")
    words = [remove_stopwords(r.split()) for r in words]
    for word in words:
      word2 = lemmatizer.lemmatize(word.lower())
      fdist[word2] += 1
    
def get_surrounding_text(chapid, senid):
    chap = next(filter(lambda x: x['id'] == chapid, book['chapters']), None)
    if chap == None:
        raise Exception("wtf")
    for i in range(0, len(chap['items'])):
        if chap['items'][i]['id'] == senid:
           return ' '.join(s['content'] for s in chap['items'][max(0,i-2):i])
    raise Exception("wtf")

# chap_sens = [{'id': chap['id'], 'sens': summarize_sentences(chap['items'], 10)} for chap in book['chapters']]


In [18]:

def get_definitions(word):
    return [s.definition() for s in wordnet.synsets(word)]

def cal_score(word):
    try:
        con = con_df.loc[word]['conc']
    except KeyError:
        con = 5
    return con * 5 + sum([1 if w.pos() == 'n' else 0 for w in wordnet.synsets(word)]) + fdist.freq(word) * 100000
    
def split_sentence(sen):
    out = re.split("[^a-zA-Z-]+",sen)
    
    if out[0] == '':
        if len(out) == 1:
            return []
        out = out[1:]
    if out[len(out) - 1] == '':
        if len(out) == 1:
            return []
        out = out[:-1]
    return out

out = Quiz()

for chap in book['chapters']:
    items = list()
    for sen in chap['items']:
        keyword = None
        ki = 0
        minScore = 9999999999999999999999
        words = split_sentence(sen['content'])
        i = -1
        for word in words:
            i += 1
            word2 = lemmatizer.lemmatize(word.lower())
            if word2 in stop_words:
                continue
            con = cal_score(word2)
            if con is not None:
                if len(get_definitions(word)) >= 3 and len(get_definitions(word)) <= 10:
                    if con < minScore:
                        minScore = con
                        keyword = word
                        ki = i 
        if keyword != None:
            anws = get_definitions(keyword)
            up = get_surrounding_text(chap["id"], sen["id"])
            ki += len(split_sentence(up))
            items.append((minScore, WordItem(str(uuid.uuid1()), up + ' ' + sen["content"], ki, anws, 0, keyword)))
#             print(keyword, minScore)
    items = [item[1] for item in sorted(items, key=lambda x: x[0])]
    if len(items) > 10:
        items = items[:10]
    
    out.chapters.append(Chapter(chap['id'], items))
#     print(chap['id'] + ' completed')

print(out)
from xml.dom import minidom
buf = minidom.parseString(out.toXML().decode('utf-8')).toprettyxml(indent="   ")
with open('book2.quiz', 'wb') as f:
  f.write(buf.encode('utf-8'))


In [13]:
for syn in wordnet.synsets("refer"):
    print(syn.definition())

make reference to
be relevant to
think of, regard, or classify under a subsuming principle or with a general group or in relation to another
send or direct for treatment, information, or a decision
seek information from
have as a meaning
use a name to designate


In [ ]:

    )


out = Quiz()
import uuid
